# KGMicrobe Animation
Sometimes, expecially when preparing a presentation for a conference or your work colleagues, a good animation can say more than a thousand words.

For this reason, we have prepared a straighforward way to create animations for a number of tasks using GRAPE that, through subsampling, can be executed on graphs of arbitrary size.

In this brief tutorial, we will show how to get a KGMicrobe embedding using DeepWalk and Walklets-based GloVe embedding, and then we will use TSNE decoposition to reduce its dimensionality and plot it into a short video.

The resulting WEBM can be converted using one of many services and can be directly incorporated in Google Slides.

## Retrieving KGMicrobe
First, we retrieve KGMicrobe:

In [1]:
from grape.datasets.kghub import KGMicrobe

graph = KGMicrobe()

Then, let's take a look at its graph report:

In [2]:
graph

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>KGMicrobe</h2><p class="paragraph">The undirected multigraph KGMicrobe has 276.36K heterogeneous nodes and 525.62K heterogeneous edges. The graph contains 26.00K connected components (of which 25.98K are disconnected nodes), with the largest one containing 249.85K nodes and the smallest one containing a single node. The RAM requirements for the nodes and edges data structures are 30.21MB and 2.54MB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 0, the maximum node degree is 11.71K, the mode degree is 1, the mean degree is 3.80 and the node degree median is 2.</p><p class="paragraph">The nodes with the highest degree centrality are microtraits.cell_shape_enum:bacillus (degree 11.71K and node type <a href='https://biolink.github.io/biolink-model/docs/AbstractEntity.html' target='_blank' title='Go to BioLink to get more informations about biolink:AbstractEntity'>biolink:AbstractEntity</a>), <a href='http://purl.obolibrary.org/obo/ECOCORE_00000173' target='_blank' title='Go to An ontology of core ecological entities to get more informations about ECOCORE:00000173'>ECOCORE:00000173</a> (degree 7.83K and node type <a href='https://biolink.github.io/biolink-model/docs/ActivityAndBehavior.html' target='_blank' title='Go to BioLink to get more informations about biolink:ActivityAndBehavior'>biolink:ActivityAndBehavior</a>), <a href='https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:24995' target='_blank' title='Go to CHEBI to get more informations about CHEBI:24995'>CHEBI:24995</a> (degree 7.34K and node type <a href='https://biolink.github.io/biolink-model/docs/ChemicalSubstance.html' target='_blank' title='Go to BioLink to get more informations about biolink:ChemicalSubstance'>biolink:ChemicalSubstance</a>), <a href='https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:52898' target='_blank' title='Go to CHEBI to get more informations about CHEBI:52898'>CHEBI:52898</a> (degree 7.25K and node type <a href='https://biolink.github.io/biolink-model/docs/ChemicalSubstance.html' target='_blank' title='Go to BioLink to get more informations about biolink:ChemicalSubstance'>biolink:ChemicalSubstance</a>) and <a href='https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:16670' target='_blank' title='Go to CHEBI to get more informations about CHEBI:16670'>CHEBI:16670</a> (degree 6.21K and node type <a href='https://biolink.github.io/biolink-model/docs/ChemicalSubstance.html' target='_blank' title='Go to BioLink to get more informations about biolink:ChemicalSubstance'>biolink:ChemicalSubstance</a>).</p><h3 style="margin: 1em 0 0 0;">Node types</h3><p class="paragraph">The graph has 11 node types, of which the 10 most common are <a href='https://biolink.github.io/biolink-model/docs/ChemicalSubstance.html' target='_blank' title='Go to BioLink to get more informations about biolink:ChemicalSubstance'>biolink:ChemicalSubstance</a> (164.72K nodes, 59.61%), <a href='https://biolink.github.io/biolink-model/docs

Since the graph contains disconnected nodes and such nodes cannot be meaningfully embedded using topological embedding, we drop them:

In [3]:
%%time
graph = graph.remove_disconnected_nodes()

CPU times: user 1.45 s, sys: 149 ms, total: 1.6 s
Wall time: 121 ms


## Compute the embedding
Next, we compute the embedding using the DeepWalk and Walklets sampling-based GloVe. Do note that this implementation is a data-race aware one that uses SGD as optimizer, and nothing fancy like ADAM or NADAM: this means that the memory footprint is only limited to the embedding size.

By default, in GRAPE we provide quite an heavy duty parametrization. For this relatively limited visualization, we can reduce it so to get the embedding a bit more quickly.

In [4]:
from grape.embedders import DeepWalkGloVeEnsmallen, WalkletsGloVeEnsmallen

In [5]:
%%time
deepwalk_embedding = DeepWalkGloVeEnsmallen(
    epochs=10,
    iterations=1,
    walk_length=128,
    window_size=5
).fit_transform(graph)

CPU times: user 29min 18s, sys: 2.6 s, total: 29min 21s
Wall time: 1min 14s


In [6]:
%%time
walklets_embedding = WalkletsGloVeEnsmallen(
    epochs=10,
    iterations=1,
    walk_length=128,
    window_size=5
).fit_transform(graph)

CPU times: user 48min 25s, sys: 6.11 s, total: 48min 31s
Wall time: 2min 5s


## Visualize the embedding on the test graph
We are at the end, finally visualizing the test graph.

In [7]:
from grape import GraphVisualizer

vis = GraphVisualizer(
    graph=graph,
    n_components=4,
    edge_embedding_method="Hadamard",
    rotate=True,
    verbose=True,
    # Automatically, since GloVe learns a cosine, the visualization tool
    # would dispatch a Cosine-distance based TSNE. This would use the sklearn
    # implementation, which is terribly slow. Therefore, we force it to use the Euclidean distance
    # and therefore the Multicore TSNE implementation (when available).
    decomposition_kwargs=dict(metric="euclidean")
)

Then we run the TSNE, this may take a while.

In [8]:
%%time
vis.fit_nodes(deepwalk_embedding)

/home/luca/anaconda3/lib/python3.9/site-packages/embiggen/visualizations/graph_visualizer.py:557: UserWarning: Please do be advised that when using a node embedding method such as Glove, which embeds nodes using a dot product, it is highly suggested to use a `cosine` metric. Using a different metric, such as the one you have provided (euclidean) may lead to worse decompositions using UMAP or t-SNE.
  warnings.warn(
Performing t-SNE using 24 cores.
Using no_dims = 4, perplexity = 30.000000, and theta = 0.500000
Computing input similarities...
Building tree...
 - point 2004 of 20000
 - point 4000 of 20000
 - point 6001 of 20000
 - point 8002 of 20000
 - point 10000 of 20000
 - point 12000 of 20000
 - point 14000 of 20000
 - point 16000 of 20000
 - point 18000 of 20000
 - point 20000 of 20000
Done in 0.00 seconds (sparsity = 0.006059)!
Learning embedding...
Iteration 51: error is 97.716087 (50 iterations in 7.00 seconds)
Iteration 101: error is 79.317347 (50 iterations in 9.00 seconds)
It

CPU times: user 9min 6s, sys: 19min 52s, total: 28min 58s
Wall time: 1min 14s


Fitting performed in 74.00 seconds.


In [9]:
%%time
vis.plot_node_types()

Rendering frames:   0%|                                                                                       …

OpenCV: FFMPEG: tag 0x30387076/'vp80' is not supported with codec id 139 and format 'webm / WebM'


Merging frames:   0%|                                                                                         …

CPU times: user 9min 48s, sys: 12.5 s, total: 10min
Wall time: 1min 29s


In [10]:
%%time
vis.fit_nodes(walklets_embedding)

Performing t-SNE using 24 cores.
Using no_dims = 4, perplexity = 30.000000, and theta = 0.500000
Computing input similarities...
Building tree...
 - point 2001 of 20000
 - point 9907 of 20000
 - point 9910 of 20000
 - point 10000 of 20000
 - point 13265 of 20000
 - point 15783 of 20000
 - point 15798 of 20000
 - point 16000 of 20000
 - point 18000 of 20000
 - point 20000 of 20000
Done in 0.00 seconds (sparsity = 0.006526)!
Learning embedding...
Iteration 51: error is 98.916224 (50 iterations in 7.00 seconds)
Iteration 101: error is 82.094338 (50 iterations in 9.00 seconds)
Iteration 151: error is 75.686108 (50 iterations in 9.00 seconds)
Iteration 201: error is 72.951000 (50 iterations in 9.00 seconds)
Iteration 251: error is 71.411492 (50 iterations in 9.00 seconds)
Iteration 301: error is 2.449860 (50 iterations in 9.00 seconds)
Iteration 351: error is 2.044232 (50 iterations in 10.00 seconds)
Iteration 400: error is 1.789491 (50 iterations in 10.00 seconds)


CPU times: user 9min 10s, sys: 19min 14s, total: 28min 24s
Wall time: 1min 13s


Fitting performed in 72.00 seconds.


In [ ]:
%%time
vis.plot_node_types()

Rendering frames:   0%|                                                                                       …